In [ ]:
%cd "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"
path = "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',None)
#不显示科学计数法，保留5位小数
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import random
from stock_portfoilo_env import *
from stable_baselines3 import *
from stable_baselines3.common.callbacks import *
from stable_baselines3.common.vec_env import *
from rl_zoo3 import linear_schedule
from torch import nn as nn
import time
import datetime

import talib
import torch
import os
import optuna
from optuna.visualization import *
from tqdm import tqdm

# Read data

In [ ]:
train_obs = np.load(path+'处理后数据\\train_obs160.npy', allow_pickle=True).astype('float32')
train_symbols = np.load(path+'处理后数据\\train_symbols160.npy', allow_pickle=True)
train_price = np.load(path+'处理后数据\\train_price160.npy', allow_pickle=True)
valid_factors = np.load(path+'数据分析\\'+'valid_factors.npy', allow_pickle=True).tolist()
stock_num = len(train_symbols[0])-1

train_obs.shape, train_symbols.shape, train_price.shape

## Data quality check
### In the notebook prepared for the data set, all positive and negative infinities and missing values were filled with zeros. Therefore, if the processed data has an unusually high number of 0 values, then it is an indication that there is a problem with the factors or a problem with the way the data was processed.

In [ ]:
zeros = []
names = []
columns = valid_factors
for i in range(train_obs.shape[-1]):
    tmp_zeros = (train_obs[np.where(train_obs[:,:,i]==0)].shape[0])
    names.append(columns[i])
    zeros.append(tmp_zeros)
    
data_quality = pd.DataFrame()
names.append('Total Zeros')
zeros.append(train_obs[np.where(train_obs==0)].shape[0])
names.append('Total Zeros Pct')
zeros.append((train_obs[np.where(train_obs==0)].shape[0])/(train_obs.flatten().shape[0]))
data_quality['name'] = names
data_quality['zeros'] = zeros

data_quality

In [ ]:
del train_obs
del train_price
del train_symbols

## Callback Function

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, log_dir, max_round, serial_num, early_stop=True, window=1, features=1, 
                 std_rolling=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__() 
        self.log_dir = log_dir
        self.best_sharpe = -np.inf
        self.best_return = -np.inf
        self.round = 0
        self.max_round = max_round
        self.early_stop = early_stop
        self.early_stop_counter = 0
        self.window = window
        self.features = features
        self.serial_num = serial_num
        self.std_rolling = std_rolling
        self.last_return = -np.inf
        self.IC_his = []
        self.lack_of_trade = False
        self.early_stop_threshold = 35
    
    def generate_random_str(self, randomlength=16):
        """
        生成一个指定长度的随机字符串
        """
        seed = int(time.time()*100000)
        random.seed(seed)
        random_str =''
        base_str ='ABCDEFGHIGKLMNOPQRSTUVWXYZabcdefghigklmnopqrstuvwxyz0123456789'
        length =len(base_str) -1
        for i in range(randomlength):
            random_str +=base_str[random.randint(0, length)]
        return random_str
    
    def _init_callback(self) -> None:
        # Create folder if needed
        if self.log_dir is not None:
            os.makedirs(self.log_dir, exist_ok=True)

    def _on_step(self) -> bool:
        done = self.locals['infos'][0]['done']
        if done == True:      
            current_time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
            print('------------------Callback Start------------------')
            print("Round", self.round, ' has finished at ', current_time, 'Early Stop Counter: ', self.early_stop_counter)
            print('#################################### Save and Test Results####################################')
            alphabet = 'abcdefghijklmnopqrstuvwxyz1234567890'
            tmp_serial_num = str(random.sample(alphabet, 10)).replace(",","").replace("'","").replace("[","").replace("]","").replace(" ","")
            tmp_test_obs = np.load(path+'处理后数据\\test_obs'+str(self.std_rolling)+'.npy', allow_pickle=True).astype('float32')[:120]
            tmp_test_price = np.load(path+'处理后数据\\test_price'+str(self.std_rolling)+'.npy', allow_pickle=True)[:120]
            tmp_test_symbols = np.load(path+'处理后数据\\test_symbols'+str(self.std_rolling)+'.npy', allow_pickle=True)[:120]
            tmp_test_obs_1 = tmp_test_obs[:,-self.window:,:]
            commission_rate = 0.00015
            slippage = 0.02
            test_env = TradingEnv(obs=tmp_test_obs_1, price=tmp_test_price, symbols=tmp_test_symbols, both_pos=True, 
                        commission_rate=commission_rate, valid_folder=None, slippage=slippage, calculate_fund=True,
                        commission=None, test=False, valid=False, active_shares=True, cash_out_stop=False, 
                        returns=1, action_type=2, num_stock=stock_num)
            obs, info = test_env.reset()
            tmp_info_1 = None
            print('#################################### Out-of-sample Results ####################################')
            for i in range(tmp_test_obs_1.shape[0]-2):
                action, _states = self.model.predict(obs, deterministic=True)
                obs, rewards, dones, truncated, info = test_env.step(action)
                if dones == True:
                    tmp_info_1 = info
                else:
                    pass
                
            return_1 = out_sample_return = tmp_info_1['return rate']
            sharpe_1 = out_sample_sharpe = tmp_info_1['sharpe']
            out_sample_long = tmp_info_1['long trades']
            out_sample_short = tmp_info_1['short trades']
                
            if out_sample_long == 0 or out_sample_short == 0:
                self.lack_of_trade = True
            else:
                self.lack_of_trade = False
                
            if out_sample_long == 0:
                long_trades_1 = 0
            else:
                long_trades_1 = out_sample_long
                
            if out_sample_short == 0:
                short_trades_1 = 0
            else:
                short_trades_1 = out_sample_short         
                
            if long_trades_1 == 0 or short_trades_1 == 0:
                self.early_stop_counter += 1
            else:
                pass
                
            if return_1 == 0 or return_1 == self.last_return:
                self.early_stop_counter += 1
            else:
                pass
                
            if return_1 <= self.last_return:
                self.early_stop_counter += 1
            else:
                pass
            
            if sharpe_1 > self.best_sharpe:
                self.best_sharpe = sharpe_1
            else:
                pass
                
            current_time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
            if out_sample_return > 0.2 and out_sample_long > 0 and out_sample_short > 0:
                tmp_serial_num = self.generate_random_str(randomlength=16)
                self.model.save(self.log_dir+tmp_serial_num+'_'+str(return_1)+'_'+current_time)
                tmp_params = np.asarray([self.window, self.features, self.std_rolling])
                np.save(self.log_dir+tmp_serial_num+'_'+str(return_1)+'.npy', tmp_params)
                print("Saving new best model to: ", self.log_dir+tmp_serial_num+'_'+str(return_1)+'_'+current_time)                       
            else:
                pass                
            if return_1 > self.best_return:
                self.best_return = return_1
                self.early_stop_counter = 0
            else:
                pass
            print("Current Best Return Rate: " , self.best_return, ', Current Best Sharpe: ', self.best_sharpe)
            self.last_return = return_1 
            self.round += 1
        else:
            pass
        
        if self.round >= self.max_round:
            continue_training = False
            try:
                os.remove(self.log_dir+self.serial_num+'.npy')
            except:
                pass
            np.save(self.log_dir+self.serial_num+'.npy', self.best_return)#self.IC_his)
        elif self.early_stop == True and self.early_stop_counter > self.early_stop_threshold:
            continue_training = False
            try:
                os.remove(self.log_dir+self.serial_num+'.npy')
            except:
                pass
            np.save(self.log_dir+self.serial_num+'.npy', self.best_return)#self.IC_his)
        else:
            continue_training = True
        if self.round >= 80 and self.best_return <= 0:
            continue_training = False
        else:
            pass
        return continue_training

In [ ]:
log_dir = path+"best_models\\"
if os.path.exists(log_dir) == True:
    pass
else:
    os.makedirs(log_dir) 

## Check the data set with default parameters
### The default parameter should be able to gradually increase the yield to within -100% without continuous non-trading.

## Parameters Optimization

In [ ]:
def generate_random_str(randomlength=16):
    """
    生成一个指定长度的随机字符串
    """
    seed = int(time.time()*100000)
    random.seed(seed)
    random_str =''
    base_str ='ABCDEFGHIGKLMNOPQRSTUVWXYZabcdefghigklmnopqrstuvwxyz0123456789'
    length =len(base_str) -1
    for i in range(randomlength):
        random_str +=base_str[random.randint(0, length)]
    return random_str

## TPE

In [ ]:
starttime = datetime.datetime.now()

In [ ]:
def objective(trial:optuna.Trial):
    torch.cuda.empty_cache()
    torch.cuda.set_device(1)
    serial_num = generate_random_str()
    print(serial_num)
    #################################################################################################################################
    STD_period = 160
    tmp_train_obs = np.load(path+'处理后数据\\train_obs'+str(STD_period)+'.npy', allow_pickle=True).astype('float32')
    tmp_train_price = np.load(path+'处理后数据\\train_price'+str(STD_period)+'.npy', allow_pickle=True)
    tmp_train_symbols = np.load(path+'处理后数据\\train_symbols'+str(STD_period)+'.npy', allow_pickle=True)
    window_size = int(trial.suggest_int("window_size", 30, tmp_train_obs.shape[1], 5))
    choose_features = True #trial.suggest_categorical("choose_features", [False, True])
    features = []
    tmp_train_obs_1 = []
    first_column = True
    for i in range(len(columns)):
        features.append(True)
    
    tmp_train_obs_1 = tmp_train_obs[:,-window_size:,:]
    print(tmp_train_obs_1.shape, tmp_train_price.shape, tmp_train_symbols.shape)
    #################################################################################################################################
    n_layers_value = trial.suggest_int("n_layers_value", 1, 5, 1)
    hidden_size_value = trial.suggest_int("hidden_size_value", 64, 512, 16)
    gamma = trial.suggest_discrete_uniform('gamma', 0.5, 0.9, 0.05)
    #gamma = trial.suggest_categorical("gamma", [0.9, 0.95, 0.98, 0.99, 0.995, 0.999, 0.9999])
    normalize_advantage = trial.suggest_categorical("normalize_advantage", [False, True])
    max_grad_norm = trial.suggest_categorical("max_grad_norm", [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 
                                                                2, 2.5, 3, 3.5, 4, 4.5, 5])
    use_rms_prop = trial.suggest_categorical("use_rms_prop", [False, True])
    gae_lambda = trial.suggest_discrete_uniform('gae_lambda', 0.5, 1, 0.05)
    #gae_lambda = trial.suggest_categorical("gae_lambda", [0.8, 0.9, 0.92, 0.95, 0.98, 0.99, 1.0])
    n_steps = trial.suggest_int("n_steps", 32, 4096, 32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    vf_coef = trial.suggest_float("vf_coef", 0, 1)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    lr_schedule = trial.suggest_categorical("lr_schedule", ["linear", "constant"])
    activation_fn_name = trial.suggest_categorical("activation_fn", ["tanh", "relu"])
    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn_name]
    #################################################################################################################################
    slippage = 0.02
    commission_rate = 0.00015
    env = TradingEnv(obs=tmp_train_obs_1, price=tmp_train_price, symbols=tmp_train_symbols, both_pos=True, 
                    commission_rate=commission_rate, valid_folder=None, slippage=slippage, calculate_fund=True,
                    commission=None, test=False, valid=False, active_shares=True, cash_out_stop=False, 
                    returns=1, action_type=1, num_stock=stock_num)
    rounds = 100
    save_and_test = True
    save_best = SaveOnBestTrainingRewardCallback(log_dir, rounds, serial_num, True, window_size, features, STD_period)
    rounds_of_train = (tmp_train_obs.shape[0])*rounds
    if lr_schedule == "linear":
        learning_rate = linear_schedule(learning_rate)
    else:
        pass
    net_pi = [hidden_size_value]*n_layers_value
    net_vf = [hidden_size_value]*n_layers_value
    policy_kwargs = dict(net_arch=dict(pi=net_pi, vf=net_vf), activation_fn=activation_fn, ortho_init=ortho_init)
    print('Now Using GPU No.', torch.cuda.current_device())
    #################################################################################################################################
    #with np.errstate(divide='ignore'):
    try:   
        model = A2C(policy='MlpPolicy', env=env, learning_rate=learning_rate, n_steps=n_steps, 
                    gamma=gamma, vf_coef=vf_coef, max_grad_norm=max_grad_norm, use_rms_prop=use_rms_prop, 
                    normalize_advantage=normalize_advantage, verbose=1, seed=123456, device='cuda', 
                    gae_lambda=gae_lambda, ent_coef=ent_coef, policy_kwargs=policy_kwargs)
        model.learn(total_timesteps=rounds_of_train, callback=save_best, log_interval=None)
        return_ = np.load(log_dir+serial_num+'.npy', allow_pickle=True)
        os.remove(log_dir+serial_num+'.npy')
        torch.cuda.empty_cache()
    except Exception as e:
        print()
        print('********************Error Info Start********************')
        print(e)
        print('********************Error Info End********************')
        print()
        return_ = -np.inf
        torch.cuda.empty_cache()
    model = None
    return return_
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=350, n_jobs=5)

In [ ]:
endtime = datetime.datetime.now()
endtime - starttime

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_slice(study, params=["STD_period", "n_layers_value", "hidden_size_value", "window_size", "n_steps"])